In [17]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import PCA
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

In [13]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

x_train_reshaped = train_x.reshape(len(train_x), 784)

pca = PCA(svd_solver='full')

X_transformed = pca.fit_transform(x_train_reshaped)

explained_variance = np.cumsum(pca.explained_variance_ratio_)

for i in range(len(explained_variance)):
  if explained_variance[i] >= 0.83:
    M = i + 1
    final_exp_var = explained_variance[i]
    break
print(f"Дисперсия необъяснённой разности для {M} компоненты: {final_exp_var}")

Дисперсия необъяснённой разности для 52 компоненты: 0.8309650832314033


In [15]:
# Ищем счёты, но зачем?
pca_M = PCA(n_components=M, svd_solver='full')
pca_M.fit(x_train_reshaped)

tr_x,  ts_x, tr_y, ts_y = train_test_split(x_train_reshaped, train_y, test_size=0.3, random_state=126)

tr_x = pca_M.transform(tr_x)
ts_x = pca_M.transform(ts_x)

print(f"Выборочное среднее нулевой колонки:", np.mean(tr_x.transpose()[0]))

Выборочное среднее нулевой колонки: 0.5965431171660649


In [16]:
forest = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=126)
clf = OneVsRestClassifier(forest).fit(tr_x, tr_y)

y_pred = clf.predict(ts_x)

cm = confusion_matrix(ts_y, y_pred)

print(f"количество верно оцененных для класса 5:", cm[5][5])

количество верно оцененных для класса 5: 1414


In [23]:
DATA = pd.read_csv("https://courses.openedu.ru/assets/courseware/v1/25c0abab1076236b7427fac937c5663e/asset-v1:ITMOUniversity+MLDATAN+spring_2022_ITMO_bac+type@asset+block/pred_for_task.csv",
                   delimiter=',',
                   index_col='FileName')
x_test = DATA.drop(['Label'], axis=1)
x_test_transformed = pca_M.transform(x_test)

y_pred = clf.predict_proba(x_test_transformed)
idx = list(DATA.index).index("file20")
print(f"вероятность, что картинка верно классифицированна", y_pred[idx][DATA['Label'][idx]])

вероятность, что картинка верно классифицированна 0.963


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but PCA was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
